In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import torch

from experiments import *
import autoencoders as ae
import wandb

In [ ]:
wandb.init(project="contrastive test")

In [ ]:
suite = ae.TrajectoryContrastiveSuite(Pendulum, contrastive_loss=ae.lifted_structured_loss(5, 10, 0.1), batch_size=256, epochs=20)
model = suite.train(80, 200)

In [ ]:
x, x_prime, dyn, cons = suite.try_transform_num(model)

In [ ]:
plt.scatter(x[:, 0], x[:, 1], c='black', s=0.01)
plt.scatter(x_prime[:, 0], x_prime[:, 1], c=dyn, s=1)

In [ ]:
# suite = ae.TrajectoryAutoencoderSuite(CoupledOscillator, additional_loss=lambda _, __ , model: model.contrastive_loss(5, 1), batch_size=512, epochs=10, is_contrastive=True)
# ree = suite.contrastive_learning(10000)
# model = ree.model

In [ ]:
# c1 = Pendulum.contrastive_data(10000)[:1000]
# c2 = Pendulum.contrastive_data(10000)[1000:2000]
# # energies = Pendulum.contrastive_data(10000)[::1000]
# #
# # ee1 = ree(ae.torch.tensor(c1).to(suite.device).float().view(-1, 3))
# # ee2 = ree(ae.torch.tensor(c2).to(suite.device).float().view(-1, 3))
# # engs = model.encoder(ae.torch.tensor(energies).to(suite.device).float().view(-1, 3)[:, 1:])
# # emb1 = model.encoder(ae.torch.tensor(c1).to(suite.device).float().view(-1, 3)[:, 1:])
# # emb2 = model.encoder(ae.torch.tensor(c2).to(suite.device).float().view(-1, 3)[:, 1:])
# # ee1_np = ee1.detach().cpu().numpy()
# # ee2_np = ee2.detach().cpu().numpy()

In [ ]:
# print(ree.C_sim, ree.C_diff)

In [ ]:
# y_true = np.linalg.norm(energies[:, 1:], axis=-1)
# y_pred = engs[:, 1].detach().numpy()

In [ ]:
# plt.scatter(y_pred, y_true)
# plt.ylabel("true E")
# plt.xlabel("predicted E")
# plt.title("Correspondence of true and predicted E")

In [ ]:
# plt.scatter(c1[:, 1], c1[:, 2], c='blue', s=1)
# plt.scatter(c2[:, 1], c2[:, 2], c='blue', s=1)
# plt.scatter(ee1_np[:, 0], ee1_np[:, 1], c='orange', s=1)
# plt.scatter(ee2_np[:, 0], ee2_np[:, 1], c=emb2[:, 0].detach().numpy(), s=1)
# plt.show()

In [ ]:
# plt.scatter(c1[:, 1], c1[:, 2], c='blue', s=1)
# plt.scatter(c2[:, 1], c2[:, 2], c='blue', s=1)
# plt.scatter(ee1_np[:, 0], ee1_np[:, 1], c=emb1[:, 0].detach().numpy(), s=1)
# plt.scatter(ee2_np[:, 0], ee2_np[:, 1], c=emb2[:, 0].detach().numpy(), s=1)
# plt.show()